# LangGraph 101

Les LLMs (Large Language Models) permettent d’intégrer de l’intelligence dans une nouvelle génération d’applications. LangGraph est un framework conçu pour aider à construire ces applications basées sur les LLMs.

Dans cette introduction, nous allons :

1. passer en revue les bases de LangGraph,

2. expliquer ses avantages,

3. montrer comment l’utiliser pour créer des workflows et des agents,

4. et enfin expliquer son fonctionnement avec LangChain et LangSmith.

![ecosystem](./img/ecosystem.png)

# Modèles de chat

Les modèles de chat sont la base des applications LLM. Ils sont généralement accessibles via une interface de chat qui prend en entrée une liste de messages et renvoie en sortie un message. **LangChain** fournit une interface standardisée pour les modèles de chat, ce qui facilite l’accès à de nombreux fournisseurs différents.

In [1]:
from dotenv import load_dotenv
load_dotenv("../.env", override=True)

True

In [3]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("openai:gpt-4o-mini", temperature=0)

## Exécution du modèle

L’interface `init_chat_model` fournit des méthodes standardisées pour utiliser les modèles de chat, notamment :

* `invoke()` : une seule entrée est transformée en une sortie.

* `stream()` : les sorties sont transmises en flux au fur et à mesure de leur production.

In [7]:
result = llm.invoke("Qu'est qu'un agent en IA?")

In [5]:
type(result)

langchain_core.messages.ai.AIMessage

In [8]:
from rich.markdown import Markdown
Markdown(result.content)

Un agent en intelligence artificielle (IA) est un système capable de percevoir son environnement, de traiter des   
informations et de prendre des décisions ou d'agir en conséquence. Les agents peuvent être simples ou complexes,   
allant de programmes informatiques qui effectuent des tâches spécifiques à des systèmes autonomes plus avancés,    
comme des robots ou des assistants virtuels.                                                                       

Voici quelques caractéristiques clés d'un agent en IA :                                                            

 1 Perception : Un agent peut recueillir des données sur son environnement à l'aide de capteurs ou d'autres moyens 
   d'entrée. Cela peut inclure des données visuelles, sonores, ou d'autres types d'informations.                   
 2 Raisonnement : Un agent utilise des algorithmes et des modèles pour analyser les données perçues, évaluer des   
   options et prendre des décisions. Cela peut impliquer des techniques d'apprentissage automatique, de logique, ou
   d'optimisation.                                                                                                 
 3 Action : Après avoir pris une décision, un agent agit sur son environnement. Cela peut impliquer des actions    
   physiques (comme un robot qui se déplace) ou des actions logiques (comme un programme qui envoie un message).   
 4 Autonomie : Certains agents sont conçus pour fonctionner de manière autonome, sans intervention humaine, tandis 
   que d'autres peuvent nécessiter une supervision ou des instructions.                                            
 5 Interaction : Les agents peuvent interagir avec d'autres agents ou avec des utilisateurs humains, ce qui peut   
   inclure la communication, la collaboration ou la compétition.                                                   

Les agents en IA sont utilisés dans de nombreux domaines, y compris la robotique, les systèmes de recommandation,  
les jeux vidéo, les assistants virtuels (comme Siri ou Alexa), et bien d'autres applications.

## Outils (Tools)

Les outils sont des utilitaires qui peuvent être appelés par un modèle de chat. Dans LangChain, la création d’outils peut se faire grâce au décorateur `@tool`, qui transforme des fonctions Python en outils appelables. Le décorateur déduit automatiquement le nom de l’outil, sa description et les arguments attendus à partir de la définition de la fonction. Il est également possible d’utiliser des serveurs **MCP (Model Context Protocol)** comme outils compatibles avec LangChain.

In [9]:
from langchain.tools import tool

@tool
def write_email(to: str, subject: str, content: str) -> str:
    """Write and send an email."""
    # Placeholder response - in real app would send email
    return f"Email sent to {to} with subject '{subject}' and content: {content}"

In [10]:
type(write_email)

langchain_core.tools.structured.StructuredTool

In [11]:
write_email.args

{'to': {'title': 'To', 'type': 'string'},
 'subject': {'title': 'Subject', 'type': 'string'},
 'content': {'title': 'Content', 'type': 'string'}}

In [12]:
Markdown(write_email.description)

Write and send an email.

## Appel d’outils (Tool Calling)

Les outils peuvent être appelés par les LLMs. Lorsqu’un outil est lié au modèle, celui-ci peut choisir de l’appeler en renvoyant une sortie structurée avec les arguments de l’outil. Nous utilisons la méthode `bind_tools` pour enrichir un LLM avec des outils.

![tool-img](img/tool_call_detail.png)

Les fournisseurs proposent souvent des paramètres comme `tool_choice` pour imposer l’appel d’outils spécifiques. L’option `any` sélectionnera au moins un des outils.

De plus, il est possible de définir `parallel_tool_calls=False` afin de s’assurer que le modèle n’appelle qu’un seul outil à la fois.

In [13]:
# Connecter des outils à un modèle de chat
model_with_tools = llm.bind_tools([write_email], tool_choice="any", parallel_tool_calls=False)

# Le modèle peut maintenant appeler les outils
output = model_with_tools.invoke("Rédige une réponse à mon patron (boss@company.ai) concernant la réunion de demain")


In [14]:
type(output)

langchain_core.messages.ai.AIMessage

In [15]:
# Extract tool calls and execute them
args = output.tool_calls[0]['args']
args

{'to': 'boss@company.ai',
 'subject': 'Réunion de demain',
 'content': "Bonjour,\n\nMerci pour l'invitation à la réunion de demain. Je serai présent et prêt à discuter des points à l'ordre du jour. Si vous avez des documents ou des informations supplémentaires à partager avant la réunion, n'hésitez pas à me les envoyer.\n\nCordialement,\n\n[Votre Nom]"}

In [17]:
output.tool_calls

[{'name': 'write_email',
  'args': {'to': 'boss@company.ai',
   'subject': 'Réunion de demain',
   'content': "Bonjour,\n\nMerci pour l'invitation à la réunion de demain. Je serai présent et prêt à discuter des points à l'ordre du jour. Si vous avez des documents ou des informations supplémentaires à partager avant la réunion, n'hésitez pas à me les envoyer.\n\nCordialement,\n\n[Votre Nom]"},
  'id': 'call_ztt1urmmAGO9bza7SOTQfzet',
  'type': 'tool_call'}]

In [16]:
# Call the tool
result = write_email.invoke(args)
Markdown(result)

Email sent to boss@company.ai with subject 'Réunion de demain' and content: Bonjour,                               

Merci pour l'invitation à la réunion de demain. Je serai présent et prêt à discuter des points à l'ordre du jour.  
Si vous avez des documents ou des informations supplémentaires à partager avant la réunion, n'hésitez pas à me les 
envoyer.                                                                                                           

Cordialement,                                                                                                      

[Votre Nom]

### Explication
Accéder aux informations d'une liste

In [18]:
tool_calls = [
    {
        'name': 'write_email',
        'args': {
            'to': 'boss@company.ai',
            'subject': 'Réunion de demain',
            'content': "Bonjour,\n\nMerci pour l'invitation à la réunion de demain. Je serai présent et prêt à discuter des points à l'ordre du jour. Si vous avez des documents ou des informations supplémentaires à partager avant la réunion, n'hésitez pas à me les envoyer.\n\nCordialement,\n\n[Votre Nom]"
        },
        'id': 'call_ztt1urmmAGO9bza7SOTQfzet',
        'type': 'tool_call'
    }
]

In [21]:
premier = tool_calls[0]['args']
print(premier)

{'to': 'boss@company.ai', 'subject': 'Réunion de demain', 'content': "Bonjour,\n\nMerci pour l'invitation à la réunion de demain. Je serai présent et prêt à discuter des points à l'ordre du jour. Si vous avez des documents ou des informations supplémentaires à partager avant la réunion, n'hésitez pas à me les envoyer.\n\nCordialement,\n\n[Votre Nom]"}


In [20]:
premier = tool_calls[0]['args']['content']
print(premier)

Bonjour,

Merci pour l'invitation à la réunion de demain. Je serai présent et prêt à discuter des points à l'ordre du jour. Si vous avez des documents ou des informations supplémentaires à partager avant la réunion, n'hésitez pas à me les envoyer.

Cordialement,

[Votre Nom]


## 

![basic_prompt](img/tool_call.png)

## Workflows

Il existe de nombreux schémas pour construire des applications avec des LLMs.

Nous pouvons intégrer des appels LLM dans des workflows prédéfinis, ce qui donne au système plus d’autonomie pour prendre des décisions.

Par exemple, nous pourrions ajouter une étape de routage pour déterminer s’il faut ou non rédiger un email.

## Agents

Nous pouvons encore augmenter l’autonomie en permettant au LLM de diriger lui-même de façon dynamique l’utilisation de ses outils.

Les agents sont généralement implémentés sous forme d’appels d’outils dans une boucle, où la sortie de chaque appel d’outil sert à informer la prochaine action.

![agent_example](img/agent_example.png)

Les agents conviennent bien aux problèmes ouverts, où il est difficile de prévoir à l’avance les étapes exactes nécessaires.

Les workflows sont souvent adaptés lorsque le flux de contrôle peut être facilement défini à l’avance.

![workflow_v_agent](img/workflow_v_agent.png)

## Qu’est-ce que LangGraph ?

LangGraph fournit une infrastructure de bas niveau qui sert de support à n’importe quel workflow ou agent.

Il n’abstrait pas les prompts ni l’architecture, et apporte plusieurs avantages :

- **Contrôle** : simplifier la définition et/ou la combinaison d’agents et de workflows.
- **Persistance** : offrir un moyen de conserver l’état d’un graphe, ce qui permet d’ajouter de la mémoire et d’intégrer l’humain dans la boucle.
- **Tests, Débogage et Déploiement** : fournir une rampe d’accès simple pour tester, déboguer et déployer des applications.

### Contrôle

LangGraph permet de définir une application comme un graphe avec :

1. **État (State)** : quelles informations devons-nous suivre tout au long de l’application ?
2. **Nœuds (Nodes)** : comment voulons-nous mettre à jour ces informations au fil de l’application ?
3. **Arêtes (Edges)** : comment voulons-nous relier ces nœuds entre eux ?

Nous pouvons utiliser la classe `StateGraph` pour initialiser un graphe LangGraph avec un objet `State`.

État (`State`) définit le schéma des informations que nous voulons suivre tout au long de l’application.

Cela peut être n’importe quel objet compatible avec `getattr()` en Python, comme un dictionnaire, une dataclass ou un objet Pydantic :

* TypedDict : le plus rapide mais ne prend pas en charge les valeurs par défaut.

* Dataclass : quasiment aussi rapide, prend en charge la syntaxe par points (state.foo) et accepte des valeurs par défaut.

* Pydantic : plus lent (surtout avec des validateurs personnalisés) mais fournit une validation stricte des types.

In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

class StateSchema(TypedDict):
    request: str
    email: str

workflow = StateGraph(StateSchema)

Chaque nœud est simplement une fonction Python ou du code TypeScript. Cela nous donne un contrôle total sur la logique à l’intérieur de chaque nœud.

Ils reçoivent l’état courant et renvoient un dictionnaire pour mettre à jour cet état.

Par défaut, les clés de l’état sont écrasées.

Cependant, il est possible de définir une logique de mise à jour personnalisée.

![nodes_edges](img/nodes_edges.png)

In [ ]:
def write_email_node(state: StateSchema) -> StateSchema:
    # Imperative code that processes the request
    output = model_with_tools.invoke(state["request"])
    args = output.tool_calls[0]['args']
    email = write_email.invoke(args)
    return {"email": email}

Les arêtes relient les nœuds entre eux.

Nous spécifions le flux de contrôle en ajoutant des arêtes et des nœuds à notre graphe d’état.

In [ ]:
workflow = StateGraph(StateSchema)
workflow.add_node("write_email_node", write_email_node)
workflow.add_edge(START, "write_email_node")
workflow.add_edge("write_email_node", END)

app = workflow.compile()

In [ ]:
app.invoke({"request": "Draft a response to my boss (boss@company.ai) about tomorrow's meeting"})

Le routage entre les nœuds peut se faire de manière conditionnelle en utilisant une simple fonction.

La valeur de retour de cette fonction est utilisée comme nom du nœud (ou liste de nœuds) vers lequel envoyer l’état suivant.

Il est également possible de fournir un dictionnaire qui associe la sortie de `should_continue` au nom du nœud suivant.

In [ ]:
from typing import Literal
from langgraph.graph import MessagesState
from email_assistant.utils import show_graph

def call_llm(state: MessagesState) -> MessagesState:
    """Run LLM"""

    output = model_with_tools.invoke(state["messages"])
    return {"messages": [output]}

def run_tool(state: MessagesState):
    """Performs the tool call"""

    result = []
    for tool_call in state["messages"][-1].tool_calls:
        observation = write_email.invoke(tool_call["args"])
        result.append({"role": "tool", "content": observation, "tool_call_id": tool_call["id"]})
    return {"messages": result}

def should_continue(state: MessagesState) -> Literal["run_tool", "__end__"]:
    """Route to tool handler, or end if Done tool called"""
    
    # Get the last message
    messages = state["messages"]
    last_message = messages[-1]
    
    # If the last message is a tool call, check if it's a Done tool call
    if last_message.tool_calls:
        return "run_tool"
    # Otherwise, we stop (reply to the user)
    return END

workflow = StateGraph(MessagesState)
workflow.add_node("call_llm", call_llm)
workflow.add_node("run_tool", run_tool)
workflow.add_edge(START, "call_llm")
workflow.add_conditional_edges("call_llm", should_continue, {"run_tool": "run_tool", END: END})
workflow.add_edge("run_tool", END)

# Run the workflow
app = workflow.compile()

In [ ]:
show_graph(app)

In [ ]:
result = app.invoke({"messages": [{"role": "user", "content": "Draft a response to my boss (boss@company.ai) confirming that I want to attend Interrupt!"}]})
for m in result["messages"]:
    m.pretty_print()

Avec ces composants de bas niveau, il est possible de construire une grande variété de workflows et d’agents. Voyez [ce tutoriel !](https://langchain-ai.github.io/langgraph/tutorials/workflows/#orchestrator-worker)

Comme les agents constituent un schéma très répandu, LangGraph propose une abstraction d’agent déjà prête.

Avec la méthode préconstruite de LangGraph, il suffit simplement de fournir le LLM, les outils et le prompt.

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=llm,
    tools=[write_email],
    prompt="Respond to the user's request using the tools provided."  
)

# Run the agent
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Rédige une réponse à mon patron (boss@company.ai) confirmant que je souhaite assister à Interrupt !"}]}
)

for m in result["messages"]:
    m.pretty_print()

### Persistance
#### Threads

Il peut être très utile de permettre aux agents de faire une pause pendant des tâches longues.

LangGraph dispose d’une couche de persistance intégrée, mise en œuvre au moyen de checkpointers, pour rendre cela possible.

Lorsque vous compilez un graphe avec un checkpointer, celui-ci enregistre un point de contrôle de l’état du graphe à chaque étape.

Les points de contrôle sont enregistrés dans un thread, qui peut être consulté une fois l’exécution du graphe terminée.


![checkpointer](img/checkpoints.png)

On compile le graphe avec un checkpointer

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

agent = create_react_agent(
    model=llm,
    tools=[write_email],
    prompt="Respond to the user's request using the tools provided.",
    checkpointer=InMemorySaver()
)

config = {"configurable": {"thread_id": "1"}}
result = agent.invoke({"messages": [{"role": "user", "content": "What are some good practices for writing emails?"}]}, config)
                    

In [ ]:
# Get the latest state snapshot
config = {"configurable": {"thread_id": "1"}}
state = agent.get_state(config)
for message in state.values['messages']:
    message.pretty_print()

In [ ]:
# Continue the conversation
result = agent.invoke({"messages": [{"role": "user", "content": "Good, let's use lesson 3 to craft a response to my boss confirming that I want to attend Interrupt"}]}, config)
for m in result['messages']:
    m.pretty_print()

In [ ]:
# Continue the conversation
result = agent.invoke({"messages": [{"role": "user", "content": "I like this, let's write the email to boss@company.ai"}]}, config)
for m in result['messages']:
    m.pretty_print()

#### Interruptions

Dans LangGraph, nous pouvons également utiliser des interruptions pour arrêter l’exécution d’un graphe à des points précis.

Cela sert souvent à recueillir une entrée auprès d’un utilisateur, puis à poursuivre l’exécution avec cette entrée collectée.

In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import InMemorySaver

class State(TypedDict):
    input: str
    user_feedback: str

def step_1(state):
    print("---Step 1---")
    pass

def human_feedback(state):
    print("---human_feedback---")
    feedback = interrupt("Please provide feedback:")
    return {"user_feedback": feedback}

def step_3(state):
    print("---Step 3---")
    pass

builder = StateGraph(State)
builder.add_node("step_1", step_1)
builder.add_node("human_feedback", human_feedback)
builder.add_node("step_3", step_3)
builder.add_edge(START, "step_1")
builder.add_edge("step_1", "human_feedback")
builder.add_edge("human_feedback", "step_3")
builder.add_edge("step_3", END)

# Set up memory
memory = InMemorySaver()

# Add
graph = builder.compile(checkpointer=memory)

In [ ]:
show_graph(graph)

In [ ]:
# Input
initial_input = {"input": "hello world"}

# Thread
thread = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread, stream_mode="updates"):
    print(event)
    print("\n")

Pour reprendre après une interruption, nous pouvons utiliser l’objet `Command`.

Nous l’utiliserons pour relancer le graphe à partir de l’état interrompu, en passant la valeur à retourner depuis l’appel d’interruption afin de poursuivre l’exécution avec `resume`

In [ ]:
# Continue the graph execution
for event in graph.stream(
    Command(resume="go to step 3!"),
    thread,
    stream_mode="updates",
):
    print(event)
    print("\n")

### Traçage

Lorsque nous utilisons LangChain ou LangGraph, la journalisation avec LangSmith fonctionne immédiatement si les variables d’environnement suivantes sont définies :

```
export LANGSMITH_TRACING=true
export LANGSMITH_API_KEY="<your-langsmith-api-key>"
```

Voici le trace LangSmith issu de l’exécution de l’agent ci-dessus :


Nous pouvons voir que l’agent est capable de poursuivre la conversation à partir de l’état précédent parce que nous avons utilisé un checkpointer.

### Déploiement

Nous pouvons également déployer notre graphe en utilisant la **LangGraph Platform.**

Cela crée un serveur **avec une API** que nous pouvons utiliser pour interagir avec notre graphe, ainsi qu’un IDE interactif appelé **LangGraph Studio.**

Il suffit de s’assurer que notre projet a une structure comme celle-ci :

```
my-app/
├── src/email_assistant   # tout le code du projet se trouve ici
│   └── langgraph101.py   # code de construction du graphe
├── .env                  # variables d’environnement
├── langgraph.json        # fichier de configuration de LangGraph
└── pyproject.toml        # dépendances du projet
```

Le fichier `langgraph.json` spécifie les dépendances, les graphes, les variables d’environnement et les autres paramètres nécessaires pour démarrer un serveur LangGraph.

Pour tester cela, déployons `langgraph_101.py`. Nous l’avons déjà déclaré dans le fichier `langgraph.json` de ce dépôt :

"langgraph101": "./src/email_assistant/langgraph_101.py:app"


**Options de déploiement avec LangGraph Platform**

Il existe plusieurs options de déploiement :

* Les déploiements locaux peuvent être lancés avec `langgraph dev` depuis le répertoire racine du dépôt. Les points de contrôle sont alors enregistrés dans le système de fichiers local.

* Il existe également différentes options [auto-hébergées.](https://docs.langchain.com/langgraph-platform/deployment-options#other-deployment-options)

* Pour les déploiements hébergés, les points de contrôle sont sauvegardés dans Postgres en utilisant un checkpointer Postgres.

Test

Demande : Rédige une réponse à mon patron (boss@company.ai
) confirmant que je souhaite assister à Interrupt !

Ici, nous pouvons voir une visualisation du graphe ainsi que l’état du graphe dans Studio.

![langgraph_studio](img/langgraph_studio.png)

Vous pouvez également consulter la documentation de l’API pour le déploiement local ici :

http://127.0.0.1:2024/docs